# PANDAS --  confidently load, explore, clean, transform, combine, and export datasets.

## DataFrame Creation

In [2]:
import pandas as pd

.read_csv("file_name") or .read_csv("path_wherefile_is") is used to read a csv file

In [3]:
r_cSv = pd.read_csv("earth.csv")

.read_excel("file_name") or .read_excel("path_wherefile_is") is used to read a excel file

In [4]:
r_Ex = pd.read_excel("water.xlsx")

## Indexing and Slicing (Loc and iloc)

In [5]:
people = pd.DataFrame({
    "Name" : ['Sumit','Sahil','Ankit'],
    "Age" : [18,19,20],
    "city" : ["uk","ny","nj"]
} , index = ['u1','u2','u3'])
print(people)

     Name  Age city
u1  Sumit   18   uk
u2  Sahil   19   ny
u3  Ankit   20   nj


In [6]:
people.loc['u1'] #row with index u1

Name    Sumit
Age        18
city       uk
Name: u1, dtype: object

In [7]:
people.loc[:,"Name"] #column by label

u1    Sumit
u2    Sahil
u3    Ankit
Name: Name, dtype: object

In [8]:
people.iloc[0:3] # it uses indexes for finding the position

,Name,Age,city
u1,Sumit,18,uk
u2,Sahil,19,ny
u3,Ankit,20,nj


In [9]:
people.iloc[:,2]

u1    uk
u2    ny
u3    nj
Name: city, dtype: object

In [10]:
people.iloc[2,:]

Name    Ankit
Age        20
city       nj
Name: u3, dtype: object

In [11]:
people.iloc[0:2,0:2]

,Name,Age
u1,Sumit,18
u2,Sahil,19


## DATA CLEANING

dropna(): remove rows/columns with missing data; supports axis, how, subset, thresh.

fillna(): replace missing values with constants or methods (ffill/bfill).

drop(): remove rows by label or columns by name.

rename(): rename columns or index labels.

isna()/notna(): detect missing values.

drop_duplicates(): remove duplicate rows.

In [23]:
import numpy as np

df = pd.DataFrame({
    "A": [1, 2, 3, np.nan, 5],
    "B": [np.nan, 2, 3, 4, 5],
    "C": [1, 2, np.nan, np.nan, 5]
})
print(df)

# 1) Detect missing
missing_map = df.isna()         # True/False per cell

# 2) Drop rows with any NaN
df_any = df.dropna()            # keep only complete rows[6]

# 3) Drop rows only if ALL values are NaN
# df.dropna(how="all")

# 4) Keep rows that have at least 2 non-NaN values
df_thresh = df.dropna(thresh=2) # requires >=2 non-NaN[6]

# 5) Fill NaN with constants or per-column mapping
filled_const = df.fillna(0)                           # fill with 0[8]
filled_map = df.fillna({"A": 0, "B": df["B"].mean(), "C": 99})
print(filled_map)
# 6) Forward/backward fill for time series
ffilled =  df.ffill(limit=2)                   # forward fill[8]
bfilled =  df.bfill(limit=1)

# 7) Drop / rename columns
clean = (df
    .drop(columns=["C"])            # remove column C
    .rename(columns={"A": "col_a", "B": "col_b"})
)
print(clean)
# 8) Remove duplicate rows
uniq = clean.drop_duplicates()
print(uniq)


     A    B    C
0  1.0  NaN  1.0
1  2.0  2.0  2.0
2  3.0  3.0  NaN
3  NaN  4.0  NaN
4  5.0  5.0  5.0
     A    B     C
0  1.0  3.5   1.0
1  2.0  2.0   2.0
2  3.0  3.0  99.0
3  0.0  4.0  99.0
4  5.0  5.0   5.0
   col_a  col_b
0    1.0    NaN
1    2.0    2.0
2    3.0    3.0
3    NaN    4.0
4    5.0    5.0
   col_a  col_b
0    1.0    NaN
1    2.0    2.0
2    3.0    3.0
3    NaN    4.0
4    5.0    5.0


## Aggregation & sorting

groupby(...).agg({...}): multi-metric aggregation.

sort_values(by=..., ascending=...): sort.

pivot_table(index=..., columns=..., values=..., aggfunc=...): reshape aggregations.

mean(), sum(), std(), median(), min(), max(): fast reducers.

In [28]:
sales = pd.DataFrame({
    "region": ["East", "East", "West", "West", "East"],
    "product": ["A", "B", "A", "B", "A"],
    "revenue": [100, 150, 200, 50, 120],
    "units": [10, 15, 20, 5, 12]
})

# Groupby with multiple aggregations
agg = sales.groupby("region").agg(
    revenue_sum=("revenue", "sum"),
    revenue_mean=("revenue", "mean"),
    units_std=("units", "std")
).reset_index()
print(agg)
print()
# Sort descending by revenue_sum
sorted_agg = agg.sort_values(by="revenue_sum", ascending=False)
print(sorted_agg)
# Direct reducers
avg_rev = sales["revenue"].mean()
std_units = sales["units"].std()

# Pivot table: revenue by region x product
pivot = pd.pivot_table(
    sales,
    index="region",
    columns="product",
    values="revenue",
    aggfunc="sum",
    fill_value=0
)
print()
print(pivot)

  region  revenue_sum  revenue_mean  units_std
0   East          370    123.333333   2.516611
1   West          250    125.000000  10.606602

  region  revenue_sum  revenue_mean  units_std
0   East          370    123.333333   2.516611
1   West          250    125.000000  10.606602

product    A    B
region           
East     220  150
West     200   50


## Merging & joining datasets

In [30]:
customers = pd.DataFrame({
    "cid": [1, 2, 3],
    "name": ["Ada", "Lin", "Sam"]
})
orders = pd.DataFrame({
    "oid": [101, 102, 103],
    "cid": [1, 2, 2],
    "amount": [250, 100, 80]
})

# Merge on key column
merged = pd.merge(customers, orders, on="cid", how="left")
print(merged)
# Concatenate rows (same columns)
more_orders = pd.DataFrame({"oid": [104], "cid": [3], "amount": [60]})
stacked = pd.concat([orders, more_orders], axis=0, ignore_index=True)

# Concatenate columns (align by index)
left_cols = customers.set_index("cid")
right_cols = orders.groupby("cid")["amount"].sum().to_frame("total_amount")
wide = left_cols.join(right_cols, how="left")


   cid name    oid  amount
0    1  Ada  101.0   250.0
1    2  Lin  102.0   100.0
2    2  Lin  103.0    80.0
3    3  Sam    NaN     NaN


In [31]:
merged.to_csv("merged_orders.csv", index=False)